In [9]:
# Import the elements you'll need to build your U-Net
import keras
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
from keras.optimizers import Adam
from keras.layers.merge import concatenate
# Set the image shape to have the channels in the first dimension
K.set_image_data_format("channels_first")


input_layer = Input(shape=(4, 160, 160, 16))
# Define a Conv3D tensor with 32 filters
down_depth_0_layer_0 = Conv3D(filters=32, kernel_size=(3,3,3),padding='same',strides=(1,1,1))(input_layer)
# Add a relu activation to layer 0 of depth 0
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)


# Create a Conv3D layer with 64 filters and add relu activation
down_depth_0_layer_1 = Conv3D(filters=64, kernel_size=(3,3,3),padding='same',strides=(1,1,1))(down_depth_0_layer_0)
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)

# Define a max pooling layer
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)

# Add a Conv3D layer to your network with relu activation
down_depth_1_layer_0 = Conv3D(filters=64, kernel_size=(3,3,3),padding='same',strides=(1,1,1))(down_depth_0_layer_pool)
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)


# Add another Conv3D with 128 filters to your network.
down_depth_1_layer_1 = Conv3D(filters=128, kernel_size=(3,3,3),padding='same',strides=(1,1,1))(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)



# Add an upsampling operation to your network
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
# Add a concatenation along axis 1

up_depth_1_concat = concatenate([up_depth_0_layer_0,down_depth_0_layer_1],axis=1)
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_1 = Conv3D(filters=64, kernel_size=(3,3,3),padding='same',strides=(1,1,1))(up_depth_1_concat)
up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)

# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_2 = Conv3D(filters=64, kernel_size=(3,3,3),padding='same',strides=(1,1,1))(up_depth_1_layer_1)
up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)

# Add a final Conv3D with 3 filters to your network.
final_conv = Conv3D(filters=3,kernel_size=(1,1,1),padding='valid',strides=(1,1,1))(up_depth_1_layer_2)

# Add a sigmoid activation to your final convolution.
final_activation = Activation('sigmoid')(final_conv)





In [10]:
# Define and compile your model
model = Model(inputs=input_layer, outputs=final_activation)
model.compile(optimizer=Adam(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy']
             )

In [11]:
# Print out a summary of the model you created
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 4, 160, 160,  0                                            
__________________________________________________________________________________________________
conv3d_24 (Conv3D)              (None, 32, 160, 160, 3488        input_5[0][0]                    
__________________________________________________________________________________________________
activation_24 (Activation)      (None, 32, 160, 160, 0           conv3d_24[0][0]                  
__________________________________________________________________________________________________
conv3d_25 (Conv3D)              (None, 64, 160, 160, 55360       activation_24[0][0]              
____________________________________________________________________________________________

In [8]:
# Import predefined utilities
import util
# Create a model using a predefined function
model_2 = util.unet_model_3d(depth=2,
                                loss_function='categorical_crossentropy',
                                metrics=['categorical_accuracy'])
# Print out a summary of the model created by the predefined function
model_2.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 4, 160, 160,  0                                            
__________________________________________________________________________________________________
conv3d_17 (Conv3D)              (None, 32, 160, 160, 3488        input_4[0][0]                    
__________________________________________________________________________________________________
activation_17 (Activation)      (None, 32, 160, 160, 0           conv3d_17[0][0]                  
__________________________________________________________________________________________________
conv3d_18 (Conv3D)              (None, 64, 160, 160, 55360       activation_17[0][0]              
____________________________________________________________________________________________